<a href="https://colab.research.google.com/github/Gaussiandra/AIJ-Junior-ruGPT-3/blob/main/generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!git clone https://github.com/sberbank-ai/ru-gpts /home/jovyan/rugpts
!pip install -r /home/jovyan/rugpts/requirements.txt

!git clone https://github.com/sberbank-ai/ruGPT3_essays /home/jovyan/gpt/ruGPT3_essays/

In [ ]:
!cp -r "/content/drive/MyDrive/ML/AIJ2020 AI4H/demo/GPT3Medium" /home/jovyan/gpt/gpt3medium/
!cp -r "/content/drive/MyDrive/ML/AIJ2020 AI4H/demo/GPT2Large" /home/jovyan/gpt/gpt2large/

---

## Generation

In [ ]:
import pexpect
import sys
import time
import os
import json
import re
import torch
import subprocess
from transformers import AutoTokenizer, AutoModel
from itertools import chain

In [ ]:
def singleton(cls):
    instances = {}
    def getinstance():
        if cls not in instances:
            instances[cls] = cls()
        return instances[cls]
    return getinstance

@singleton
class Similarity:
    def __init__(self):
        self.model_name = 'DeepPavlov/rubert-base-cased'
        self.model = AutoModel.from_pretrained(self.model_name)
        self.tokenizer = AutoTokenizer.from_pretrained(self.model_name)

        self.device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
        self.model.eval()
        self.model.to(self.device)
    
    def get_most_similar(self, theme, essays):
        with torch.no_grad():
            embeddings = torch.empty(0).to(self.device)
            for text in chain([theme], essays):
                tokenized_text = self.tokenizer.encode_plus(text, return_tensors='pt')
                input_ids, _, attention_mask = tokenized_text.values()
                cur_emb = self.model(input_ids.to(self.device), attention_mask.to(self.device))[1]
                embeddings = torch.cat([embeddings, cur_emb], dim=0)

            sim = torch.cosine_similarity(embeddings[0].repeat(len(essays), 1), embeddings[1:])
            index = sim.argmax()
            return essays[index]

In [ ]:
def get_essay(subject, theme, child, n_attempts=2):
    subs = {'russian': 'рус',
            'social': 'общ',
            'history': 'ист',
            'literature': 'лит'}
    pattern = 'ПРЕД {} ТЕКСТ {} СОЧИН '
    
    essays = []
    for _ in range(n_attempts):
        cur_essay = ' '
        while (cur_essay[-1] not in '.?!…' or 
               cur_essay.count(' ') < 175 or 
               'СОЧИН' in cur_essay):
            model_prompt = pattern.format(subs[subject], theme)
            model_prompt = re.sub('\(\d+\)', '', model_prompt)

            child.sendline(model_prompt)
            child.expect(['Context'], timeout=240)

            model_output = child.before.decode('utf-8')
            essay_first_idx = model_output.rfind(model_prompt) + len(model_prompt) + 1
            cur_essay = model_output[essay_first_idx:-2]   

        essays.append(cur_essay)
    
    if n_attempts > 1:
        return Similarity().get_most_similar(theme, essays)
    else:
        return essays[0]

#### GPT2Large

In [ ]:
%%writefile /home/jovyan/rugpts/scripts/generate_ruGPT2Large.sh
#! /bin/bash

NUM_GPUS_PER_WORKER=1

mpirun --allow-run-as-root --np ${NUM_GPUS_PER_WORKER} python /home/jovyan/rugpts/generate_transformers.py \
    --model_type=gpt2 \
    --model_name_or_path=/home/jovyan/gpt/gpt2large/ \
    --p=0.9 \
    --k=7 \
    --length=700 \
    --num_return_sequences=1 \
    --repetition_penalty=1.2 \

Overwriting /home/jovyan/rugpts/scripts/generate_ruGPT2Large.sh


In [ ]:
gpt2_child = pexpect.spawn("bash /home/jovyan/rugpts/scripts/generate_ruGPT2Large.sh")
gpt2_child.expect('Context', timeout=None)
gpt2_child.logfile = open('/home/jovyan/gpt/mylog.txt', 'wb')

In [ ]:
get_essay(
    'history', 
    'Период истории России 1925-1943 гг. (индустриализация и коллективизация страны)',
    gpt2_child,
    1
)

'Историки оценивают период 1924—1953 годов неоднозначно, отмечая как индустриализацию, так и масштабные репрессии против «врагов народа». В целом историки признают необходимость форсированной модернизации советской экономики в интересах скорейшего восстановления народного хозяйства после гражданской войны и усиления военного потенциала страны. Вместе с тем многие из них подчеркивают непоследовательность политики Советского правительства по отношению к крестьянству. С одной стороны, государство проводило политику расширения закупок сельскохозяйственной продукции у населения, стремилось привлечь иностранные инвестиции; были осуществлены первые массовые кампании по борьбе со взяточничеством на селе; развернулось активное строительство жилья для рабочих городов. Другой стороной медали стал массовый террор периода 30-х — 40-х годов прошлого века: необоснованные аресты, выселения целых деревень, массовое переселение семей «кулаков» за границу.'

In [ ]:
gpt2_child.close()